In [149]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import emcee
import corner
%matplotlib widget

### Definition of the Matern 5/2 Kernel and its derivatives

In [150]:
def matern52(t1, t2, *, lnsigma = np.log(1.0), lnrho = np.log(3.0)):
    rho = np.exp(lnrho)
    sigma = np.exp(lnsigma)
    x = np.sqrt(5) * np.abs(t1 - t2) / rho
    return sigma ** 2 * (1 + x + x ** 2 / 3) * np.exp(-x)

def matern52_cross(t1, t2, *, lnsigma = np.log(1.0), lnrho = np.log(3.0)):
    rho = np.exp(lnrho)
    sigma = np.exp(lnsigma)
    x = np.sqrt(5) * np.abs(t1 - t2) / rho
    return np.sqrt(5) * sigma ** 2 * np.sign(t1 - t2) * (x + x ** 2) / (3 * rho) * np.exp(-x)

def matern52_grad(t1, t2, *, lnsigma = np.log(1.0), lnrho = np.log(3.0)):
    rho = np.exp(lnrho)
    sigma = np.exp(lnsigma)
    x = np.sqrt(5) * np.abs(t1 - t2) / rho
    return (5 / 3) * sigma ** 2 * (1 + x - x ** 2) / rho ** 2 * np.exp(-x)

def sample_gp(random, K, size=None):
    return random.multivariate_normal(np.zeros(K.shape[0]), K, size=size)

### Definition of the diagonal and off-diagonal terms for the covariance block matrix

In [151]:
def K_11(t_1, t_2, p):
    if isinstance(t_1, np.ndarray) and isinstance(t_2, np.ndarray):
        t_1 = t_1[:, None]
        t_2 = t_2[None, :]
        
    A, B, C, D, lnrho = p
    
    # Note that the 2nd and 3rd terms cancel by a sign
    first_term = A**2 * matern52(t_1, t_2, lnrho=lnrho)
    #second_term = A*B * matern52_cross(t_f[:, None], t_f[None, :])
    #third_term = -B*A * matern52_cross(t_f[:, None], t_f[None, :])
    fourth_term = B**2 * matern52_grad(t_1, t_2, lnrho=lnrho)
    
    return first_term + fourth_term
    
def K_12(t_1, t_2, p):
    
    if isinstance(t_1, np.ndarray) and isinstance(t_2, np.ndarray):
        t_1 = t_1[:, None]
        t_2 = t_2[None, :]
    
    A, B, C, D, lnrho = p
    
    first_term = A*C * matern52(t_1, t_2, lnrho=lnrho)
    second_term = A*D * matern52_cross(t_1, t_2, lnrho=lnrho)
    third_term = B*C * matern52_cross(t_2, t_1, lnrho=lnrho)
    fourth_term = B*D * matern52_grad(t_1, t_2, lnrho=lnrho)
    
    return first_term + second_term + third_term + fourth_term

def K_21(t_1, t_2, p):
    
    if isinstance(t_1, np.ndarray) and isinstance(t_2, np.ndarray):
        t_1 = t_1[:, None]
        t_2 = t_2[None, :]
    
    A, B, C, D, lnrho = p
    
    first_term = C*A * matern52(t_1, t_2, lnrho=lnrho)
    second_term = C*B * matern52_cross(t_1, t_2, lnrho=lnrho)
    third_term = D*A * matern52_cross(t_2, t_1, lnrho=lnrho)
    fourth_term = D*B * matern52_grad(t_1, t_2, lnrho=lnrho)
    
    return first_term + second_term + third_term + fourth_term
    
def K_22(t_1, t_2, p):
    
    if isinstance(t_1, np.ndarray) and isinstance(t_2, np.ndarray):
        t_1 = t_1[:, None]
        t_2 = t_2[None, :]
    
    A, B, C, D, lnrho = p
    
    # Note the 2nd and 3rd terms cancel by a sign
    first_term = C**2 * matern52(t_1, t_2, lnrho=lnrho)
    #second_term = C*D * matern52_cross(t_rv[:, None], t_rv[None, :])
    #third_term = -D*C * matern52_cross(t_rv[:, None], t_rv[None, :])
    fourth_term = D**2 * matern52_grad(t_1, t_2, lnrho=lnrho)
    
    return first_term + fourth_term

### Load in time series data, use only certain sections

In [152]:
var_path = "example data/tau_0.050"

t_flux = np.load(var_path + "_t.npy")[:750]
flux = np.load(var_path + "_f.npy")[:750]
flux_err = np.load(var_path + "_ferr.npy")[:750]

mu = np.mean(flux)
flux = (flux / mu - 1) * 1e3
flux_err = flux_err * 1e3 / mu

t_rad = np.load(var_path + "_t.npy")[450:1000]
rv = np.load(var_path + "_rv.npy")[450:1000]
rv_err = np.load(var_path + "_rverr.npy")[450:1000]

In [153]:
def cov_mat(params, t_f, t_rv):
    """
    function to build covariance matrix
    """
    Kappa11 = K_11(t_f, t_f, params)
    Kappa12 = K_12(t_f, t_rv, params)
    Kappa21 = Kappa12.T
    Kappa22 = K_22(t_rv, t_rv, params)

    cov = np.concatenate((
          np.concatenate((Kappa11, Kappa12), axis=1),
          np.concatenate((Kappa21, Kappa22), axis=1),
          ), axis=0)
    
    return cov

In [154]:
def log_like(r, K):
    """
    Pulled from Dan's notebook, updated with Cholesky decomposition
    https://github.com/dfm/gp/blob/main/solutions.ipynb
    
    The multivariate Gaussian ln-likelihood (up to a constant) for the
    vector ``r`` given a covariance matrix ``K``.
    
    :param r: ``(N,)``   The residual vector with ``N`` points.
    :param K: ``(N, N)`` The square (``N x N``) covariance matrix.
    
    :returns lnlike: ``float`` The Gaussian ln-likelihood. 
    
    """
    # Slow version, factor ~2x slower.
    #return -0.5 * (np.dot(r, np.linalg.solve(K, r)) + np.linalg.slogdet(K)[1])

    # Cholesky decomposition, faster
    # For more info, check out: https://math.stackexchange.com/questions/3158303/using-cholesky-decomposition-to-compute-covariance-matrix-determinant
    try:
        cho_decomp = scipy.linalg.cho_factor(K)
        log_det_cov = 2*np.sum(np.log(np.diag(cho_decomp[0])))
        return -0.5 * (np.dot(r, scipy.linalg.cho_solve(cho_decomp, r)) + log_det_cov)# + (len(r)*np.log(2.*np.pi)))
    except np.linalg.LinAlgError:
        return -np.inf

In [155]:
def gp_log_prob(params, t_f, t_rv, y, y_err):
    """
    add in mean flux and mean RVs as potential parameters in the model
    
    Hard uniform bounds for coefficients
    """
    
    b = [(0.01, 10.0), (-3.0, 3.0), (-250.0, 250.0), (-250.0, 250.0), np.log((0.1, 5)),
         (0.01, 10.0), (-3.0, 3.0), (-250.0, 250.0), (-250.0, 250.0), np.log((0.1, 5))]
    
    for b, p in zip(bounds, params):
        if p < b[0] or p > b[-1]:
            return -np.inf
    
    # Compute the covariance matrix for the first GP
    K1 = cov_mat(params[:5], t_f, t_rv)
    
    # Compute the covariance matrix for the second GP
    K2 = cov_mat(params[5:], t_f, t_rv)
    
    K = K1 + K2
    K[np.diag_indices_from(K)] += y_err**2
    
    # Compute the log likelihood
    return log_like(y, K)

def gp_neg_log_prob(params, t_f, t_rv, y, y_err):
    
    # Compute the covariance matrix for the first GP
    K1 = cov_mat(params[:5], t_f, t_rv)
    
    # Compute the covariance matrix for the second GP
    K2 = cov_mat(params[5:], t_f, t_rv)
    
    K = K1 + K2
    K[np.diag_indices_from(K)] += y_err**2
    
    # Compute the negative log likelihood
    return -log_like(y, K)

### Optimize model parameters and Sample Parameters

In [156]:
def minimize_gp_kernel(y):
    
    p0 = np.array([0.5, -0.4, 0.7, 5.0, np.log(2.3),
                   0.5, -0.4, 0.7, 5.0, np.log(2.3)])
    
    b = [(0.01, 10.0), (-3.0, 3.0), (-250.0, 250.0), (-250.0, 250.0), np.log((0.1, 5)),
         (0.01, 10.0), (-3.0, 3.0), (-250.0, 250.0), (-250.0, 250.0), np.log((0.1, 5))]
    
    #options={'disp': None, 'maxls': 20, 'iprint': -1, 'gtol': 1e-08, 'eps': 1e-08, 'maxiter': 15000, 'ftol': 1e-09, 'maxcor': 10, 'maxfun': 20000}
    options = {'maxiter':25000}
    
    result = scipy.optimize.minimize(gp_neg_log_prob, p0, args=(t_flux, t_rad, y, np.concatenate((flux_err, rv_err))), method='Nelder-Mead', bounds=b, options=options)
    
    return result

In [157]:
y = np.concatenate((flux, rv))
res = minimize_gp_kernel(y)

print(res)

 final_simplex: (array([[ 1.91953491e+00, -1.79801451e+00,  2.34642056e+00,
         7.78593009e+01,  1.60943791e+00,  6.83074296e+00,
        -2.33352641e-01,  7.13428565e+00, -2.49932386e+02,
         1.54207894e+00],
       [ 1.91953560e+00, -1.79801458e+00,  2.34642810e+00,
         7.78592817e+01,  1.60943791e+00,  6.83074290e+00,
        -2.33352679e-01,  7.13428063e+00, -2.49932386e+02,
         1.54207895e+00],
       [ 1.91953434e+00, -1.79801451e+00,  2.34642428e+00,
         7.78593017e+01,  1.60943791e+00,  6.83074287e+00,
        -2.33352806e-01,  7.13427705e+00, -2.49932386e+02,
         1.54207893e+00],
       [ 1.91953308e+00, -1.79801442e+00,  2.34641697e+00,
         7.78593809e+01,  1.60943791e+00,  6.83074378e+00,
        -2.33352967e-01,  7.13427915e+00, -2.49932386e+02,
         1.54207897e+00],
       [ 1.91953362e+00, -1.79801447e+00,  2.34642130e+00,
         7.78593287e+01,  1.60943791e+00,  6.83074344e+00,
        -2.33353003e-01,  7.13427692e+00, -2.49932386

In [158]:
bounds = [(0.01, 10.0), (-3.0, 3.0), (-250.0, 250.0), (-250.0, 250.0), np.log((0.1, 5)),
         (0.01, 10.0), (-3.0, 3.0), (-250.0, 250.0), (-250.0, 250.0), np.log((0.1, 5))]

ndim = 5*2
nwalkers = 250
pos = res.x

pos_fixed = []

for b, p in zip(bounds, pos):
    if p < b[0] + 0.1:
        pos_fixed.append(p + 0.5)
    elif p > b[-1] - 0.1:
        pos_fixed.append(p - 0.5)
    else:
        pos_fixed.append(p)

pos = pos + 1e-3*np.random.randn(nwalkers, ndim)
sampler = emcee.EnsembleSampler(nwalkers, ndim, gp_log_prob,
                                args=(t_flux, t_rad, np.concatenate((flux, rv)), np.concatenate((flux_err, rv_err))))

#sampler.run_mcmc(pos, 1000, progress=True);

In [159]:
l = ["A", "B", "C", "D", r"log$(\rho_1)$",
     "E", "F", "G", "H", r"log$(\rho_2)$"]

#np.save("example data/two_latent_GP_actual_data_samples.npy", sampler.flatchain)
#np.save("example data/two_latent_GP_mcmc_samples_full_model_n7500_optimized.npy", sampler.flatchain)

#corner.corner(sampler.flatchain[-1000:], labels=l)

#corner.corner(samples, labels=["A", "B", "C", "D", r"$\sigma$", r"$\rho$"], truths=[1.0, -5.0, 0.9, 12.5, np.log(4.0), np.log(2.5)])

In [160]:
#samples = np.load("example data/two_latent_GP_mcmc_samples_full_model_n7500_optimized.npy")
#corner.corner(samples[25000:], labels=l, truths=p_test)

### Plot model and data

In [161]:
p0 = res.x
#print(res.x)
#t_flux_hold = np.concatenate((t_flux, [max(t_flux) + 1]))

cov_train1 = cov_mat(p0[:5], t_flux, t_rad)
cov_train2 = cov_mat(p0[5:], t_flux, t_rad)
cov_train = cov_train1 + cov_train2

In [164]:
y = np.concatenate((flux, rv))
t = np.concatenate((t_flux, t_rad))

factor = (scipy.linalg.cholesky(cov_train, overwrite_a=True, lower=False), False)
alpha  = scipy.linalg.cho_solve(factor, y, overwrite_b=True)

t_test_flux = np.linspace(min(t_flux), max(t_flux), 1000)
t_test_rv  = np.linspace(min(t_rad), max(t_rad), 500)

k_test_flux = np.zeros((len(t_test_flux)))
k_test_rv = np.zeros((len(t_test_rv)))

for i in range(len(t_test_flux)):
    
    flux_cov1_f = K_11(t_test_flux[i], t_flux, p0[:5])
    flux_cov1_g = K_11(t_test_flux[i], t_flux, p0[5:])
    flux_cov2_f = K_12(t_test_flux[i], t_rad, p0[:5])
    flux_cov2_g = K_12(t_test_flux[i], t_rad, p0[5:])
    
    flux_cov1 = flux_cov1_f + flux_cov1_g
    flux_cov2 = flux_cov2_f + flux_cov2_g
    
    k_test_flux[i] = np.dot(np.concatenate((flux_cov1, flux_cov2)), alpha)
    
for j in range(len(t_test_rv)):
    
    rv_cov2_f = K_22(t_test_rv[j], t_rad, p0[:5])
    rv_cov2_g = K_22(t_test_rv[j], t_rad, p0[5:])
    
    rv_cov1_f = K_21(t_test_rv[j], t_flux, p0[:5])
    rv_cov1_g = K_21(t_test_rv[j], t_flux, p0[5:])
    
    rv_cov2 = rv_cov2_f + rv_cov2_g
    rv_cov1 = rv_cov1_f + rv_cov1_g
    
    k_test_rv[j] = np.dot(np.concatenate((rv_cov1, rv_cov2)), alpha)

In [165]:
def plot_data_and_model(p0=None):
    
    fig, ax = plt.subplots(nrows=2, figsize=(12, 6), gridspec_kw={'hspace':0.2})
    
    ax[0].scatter(t_flux, flux, color='blue', s=20.0, alpha=0.5)
    ax[1].scatter(t_rad, rv, color='orange', s=20.0, alpha=0.5)
    
    ax[0].plot(t_test_flux, k_test_flux, lw=2.0, color='black', ls='-', zorder=1)
    ax[1].plot(t_test_rv, k_test_rv, lw=2.0, color='black', ls='-', zorder=1)
    #ax[1].plot(t_rad, y_samp[len(t_flux):], lw=2.0, color='orange', ls='--')
    
    #ax[0].plot(t_flux, mod_f, lw=2.0, color='black', ls='--')
    #ax[1].plot(t_rad, mod_rv, lw=2.0, color='black', ls='--')
    
    ax[0].set_ylabel(r"Norm. Flux (ppt)")
    ax[1].set_ylabel(r"RV (m s$^{-1}$)")
    
    ax[1].set_xlabel(r"Time (day)")
    
    ax[0].set_xlim([min(t_flux), max(t_flux)])
    ax[1].set_xlim([min(t_rad), max(t_rad)])
    
    #plt.savefig("example data/two_latent_GP_model_comparison.png")
    
    plt.tight_layout()
    plt.show()
    
#plot_data_and_model(p0 = np.array([0.1, 0.04, 0.07, 5.0, 20.0, 2.0]))
plot_data_and_model()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/var/folders/pp/slx5z51s7j9d_w8l_mb2blvc0001sl/T/ipykernel_42997/2219638672.py:25: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
